In [1]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

## memory 공유 확인 위한 dummy chain 생성

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dummy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [4]:
dummy_chain = dummy_prompt | model 

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory

dumy_chain_with_memory = RunnableWithMessageHistory(
    dummy_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [6]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 취소하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 1e96be89-14e1-4c49-8649-8d0d507c315b not found for run acb24f44-956a-4736-b9f1-18cb059ea239. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b070c956-46c6-4715-a4c3-58e60116059c-0')

## 주문 변경/취소 여부 분류 
* RunnableWithMessageHistory 사용 X

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [9]:
classify_change_or_cancel_chain = order_change_cancel_prompt | model
classify_change_or_cancel_chain

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000018DF22CE4D0>, asy

In [12]:
chat_history = store["test_240516-1"].messages
chat_history 

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b070c956-46c6-4715-a4c3-58e60116059c-0')]

In [13]:
response = classify_change_or_cancel_chain.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)
response

AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 87, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d34eb9c1-038e-4d55-ae90-ccae9b44c59e-0')

## 요청 or 승인 여부 분류

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

classify_confirmation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.
            
            To make this determination, the following conditions must be met:
            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.
            2. The user must have explicitly expressed consent in response to the AIMessage asking for approval of the action specified in action_type.
            
            Your response must be either 'yes' or 'no'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}\naction_type: {action_type}"),
    ]
)
classify_confirmation_prompt

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [16]:
classify_confirmation_chain = classify_confirmation_prompt | model 
classify_confirmation_chain

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [17]:
exp_chain = {
    "action_type": classify_change_or_cancel_chain
} | classify_confirmation_chain
exp_chain

{
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
               | ChatOpenAI(client=<openai.resources.chat.completions.Completions ob

In [18]:
exp_chain.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)

KeyError: "Input to ChatPromptTemplate is missing variables {'input', 'chat_history', 'order_id'}.  Expected: ['action_type', 'chat_history', 'input', 'order_id'] Received: ['action_type']"

chat_history 때문에 발생하는 오류일 것

In [22]:
exp_chain2 = (
    {
    "chat_history": chat_history,
    "input": None,
    "order_id": 7,
    "action_type": classify_change_or_cancel_chain
} | classify_confirmation_chain
)
exp_chain2

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>

In [24]:
from langchain_core.runnables import  RunnablePassthrough

exp_chain3 = {
    "chat_history": RunnablePassthrough(),
    "input": RunnablePassthrough(),
    "order_id":RunnablePassthrough(),
    "action_type": classify_change_or_cancel_chain
} | classify_confirmation_chain

exp_chain3

{
  chat_history: RunnablePassthrough(),
  input: RunnablePassthrough(),
  order_id: RunnablePassthrough(),
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\nord

In [25]:
exp_chain3.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)

ValueError: variable chat_history should be a list of base messages, got {'chat_history': [HumanMessage(content='주문 취소하고 싶어'), AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b070c956-46c6-4715-a4c3-58e60116059c-0')], 'input': None, 'order_id': 7}

In [26]:
exp_chain3.invoke(
    {"chat_history": Rulambda x: x["chat_history"],
     "input": None,
     "order_id": 7}
)

ValueError: variable chat_history should be a list of base messages, got <function <lambda> at 0x0000018DF2339BD0>

In [27]:
from langchain_core.memory import ChatMessageHistory

ImportError: cannot import name 'ConversationBufferMemory' from 'langchain_core.memory' (c:\Users\Tae-su\Grow\repositories\orderbot_v2\venv\lib\site-packages\langchain_core\memory.py)

In [28]:
exp_chain4 = {"action_type": classify_change_or_cancel_chain} | RunnablePassthrough()
exp_chain4

{
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
               | ChatOpenAI(client=<openai.resources.chat.completions.Completions ob

In [29]:
exp_chain4.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)

{'action_type': AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 87, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0a5863cf-6a79-4900-b8d6-2ec502ac8f09-0')}

In [42]:
def helper(dict):
    input = dict["pass"]
    action_type =  dict["action_type"]
    action_type = action_type.content
    input["action_type"] = action_type
    return input

In [40]:
from langchain_core.runnables import RunnableLambda

exp_chain5 = (
    {
        "pass": RunnablePassthrough(),
        "action_type": classify_change_or_cancel_chain
    } | helper
    )
exp_chain5

TypeError: unsupported operand type(s) for |: 'dict' and 'function'

In [43]:
from langchain_core.runnables import RunnableLambda

exp_chain5 = (
    {
        "pass": RunnablePassthrough(),
        "action_type": classify_change_or_cancel_chain
    } | RunnableLambda(helper)
    )
exp_chain5

{
  pass: RunnablePassthrough(),
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
               | ChatOpenAI(client=<openai.resources.

In [44]:
exp_chain5.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)

{'chat_history': [HumanMessage(content='주문 취소하고 싶어'),
  AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b070c956-46c6-4715-a4c3-58e60116059c-0')],
 'input': None,
 'order_id': 7,
 'action_type': '주문 취소'}

In [45]:
exp_chain6 = exp_chain5 | classify_confirmation_chain
exp_chain6

{
  pass: RunnablePassthrough(),
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])
               | ChatOpenAI(client=<openai.resources.

In [47]:
exp_chain6.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
).content

'no'

## langchain bot 답변

In [ ]:
from langchain_core.runnables import RunnableWithMessageHistory

assign_confirmation_runnable = RunnableWithMessageHistory(
    input_messages={
        "chat_history": MessagesPlaceholder(variable_name="chat_history"),
        "input": MessagesPlaceholder(variable_name="input"),
        "order_id": MessagesPlaceholder(variable_name="order_id"),
        "action_type": MessagesPlaceholder(variable_name="action_type"),
    },
    memory=ConversationBufferMemory(),
    callable=lambda x: {
        "chat_history": x["chat_history"],
        "input": x["input"],
        "order_id": x["order_id"],
        "action_type": x["action_type"],
        "confirmation": classify_confirmation_chain.invoke(x),
    },
)

# Now you can invoke the assign_confirmation_runnable
output = assign_confirmation_runnable.invoke(
    {
        "chat_history": chat_history,
        "input": None,
        "order_id": 7,
        "action_type": "주문 변경",  # Replace with the actual action_type value
    }
)

# The output will be a dictionary with the required keys, including "confirmation"
print(output)